# CE-40959: Deep Learning

In [ ]:
# importing modules
from utils import load_data
from models import Dense
from train import train
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# The following two lines let us reload external modules in the notebook
%load_ext autoreload
%autoreload 2

# Data Description and Loading Data

On this notebook, we are going to work on farsi OCR dataset. As its name implies, it is like famous **MNIST** dataset but it consists of images of handwritten digits in farsi. Each instance of this dataset is 32 * 32 gray-scale image. It is totally composed of 80000 instances. After loading this data, let's plot some images in order to see how they look like.

Train, validation and test sets are loaded using a method in `utils.py`. Training set includes 0.7 of the whole dataset and test set just has 0.1 of it. Rest is assigned as validation set.

**Note**: Images are flattened that's why their size is 1024 = 32 * 32.

In [ ]:
x_train, y_train, x_validation, y_validation, x_test, y_test = load_data()

In [ ]:
num_sample = 10
num_class = 10
fig, ax = plt.subplots(nrows=num_sample, ncols=num_class)

for i in range(num_class):
    class_i_images = [x_train[k] for k in range(x_train.shape[0]) if y_train[k][i] == 1]
    for j in range(num_sample):
        ax[j, i].get_xaxis().set_visible(False)
        ax[j, i].get_yaxis().set_visible(False)
        ax[j, i].imshow(class_i_images[j].reshape((32, 32)))

# Setting Flags
Tensorflow has the option of setting of some flags which should be defined once and can be used in any file later.

Here we define flags for
1. `learning_rate`: Shows the initial learning rate in optimization process.

2. `num_epoch`: The total number of epochs for training process.

3. `weight_decay`: The coeffecient of L2 Loss term in total Loss function.

4. `batch_size`: Size of each batch given to model.

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('num_epoch', 10, 'Number of epochs to train.')
flags.DEFINE_float('weight_decay', 0., 'Weight for L2 loss')
flags.DEFINE_integer('batch_size', 100, 'Size of batch')

# Defining Initializers

In the following cell we are importing some intializers which are defined in `utils.py`. Based on their explanation, you have to complete their code in `utils.py` and then import them.

In [ ]:
from utils import glorot_initializer, zero_initializer, normal_initializer

# Developing MLP Model

`Dense` model is defined as a python class in `models.py`. Its constructor receives as input followings:

1. num_hidden: Number of hidden units and output units.
2. weight_initializer: Function used for initializing weights
3. bias_initializer: Function used for initializing biases
4. act: Activation function used for hidden layers
5. logging: This is a boolean showing whether the model saves log of weights and biases for later visualization using tensorboard.
6. stddev: Standard deviation in case of having normal initializer for weights of layers.

You have to complete some parts of `__init__`, `_loss`, `_accuracy`, `_log_vars` and `_build` methods in this class.

There is also another python file `layers.py` consists of the class `DenseLayer`. The arguments of its constructor are:
1. input_dim: Dimension of input to layer
2. output_dim: Dimension of output of layer
3. act: Activation function of layer
4. weight_initializer: Function used for initializing weights
5. bias_initializer: Function used for initializing biases
6. stddev: Standard deviation in case of having normal initializer for weights of the layer.

Some part of `__call__` method of is left for you to complete.

**Note**: It is necessary to complete aforementioned python files before moving forward to following cells.

# Experimenting on features of an MLP

In the rest of this notebook, some experiments should be done on different setting like regularization, activation function, number of layers, etc combined with some visualizations.

## Different activation functions

On the following cell we have defined a model with number of hidden units 200, 100, 50, 10. Note that the outputs of last 10 units, after which a softmax function is applied, act as scores for 10 class of digits in data.

You also have to use uniform glorot initializer and zero initializer for weights and biases, respectively.

The last point here is that we are using **sigmoid** as activation function of all layers.

Later, we will apply another one and observe its difference

In [ ]:
model = Dense(num_hidden=[200, 100, 50, 10],
              weight_initializer=glorot_initializer,
              bias_initializer=zero_initializer,
              act=tf.nn.sigmoid,
              logging=True)

After defining the model we need to trigger training process by using the code partly prepared in `train.py`. After filling required lines run the following cell to start training.
In this file some writers are defined which are later used for plotting visualizations in tensorboard framework. Summary information defined as scalars (like loss) and histograms (like weights) are saved by this writers in `logs` folder near existing files. More specifically, for each model another folder whose name came from `log_file` variable is created.

Furthermore, when you define a session using `with`, the session is just restricted to its following context and can not be used in outer scope.

In [ ]:
with tf.Session() as sess:
    log_file = str(model.act.__name__)
    train(x_train, y_train, x_validation, y_validation, x_test, y_test, model, sess, log_file)

**Important Note:** In order to run different computation graphs in the same process, it is compulsary to reset the default graph to a new one which is defined later. To do so, you have to use method `tf.rest_default_graph()` as in the following cell. Otherwise, you will end up with lots of nodes in the default graph of computation.

In [ ]:
tf.reset_default_graph()

Let's maintain the same settings but change activation function to **tanh** and run the whole process again.

In [ ]:
model = Dense(num_hidden=[200, 100, 50, 10],
              weight_initializer=glorot_initializer,
              bias_initializer=zero_initializer,
              act=tf.nn.tanh,
              logging=True)
with tf.Session() as sess:
    log_file = str(model.act.__name__)
    train(x_train, y_train, x_validation, y_validation, x_test, y_test, model, sess, log_file)
tf.reset_default_graph()

---

# Question 1
Compare `tanh` and `sigmoid` based on above results. Explain your observation from the visualizations produced by tensorboard.

---

## Different Initializations

Next we want to see the effect of aforementioned initializers.

At first we use zero initializer both for biases and weights. By doing so, specially for weight initialization, the network will get in trouble with breaking the symmetry.

In [ ]:
model = Dense(num_hidden=[200, 50, 10],
              weight_initializer=zero_initializer,
              bias_initializer=zero_initializer,
              act=tf.nn.tanh,
              logging=True)

with tf.Session() as sess:
    log_file = str(model.act.__name__) + "_" + str(model.weight_initializer.__name__)
    train(x_train, y_train, x_validation, y_validation, x_test, y_test, model, sess, log_file)
tf.reset_default_graph()    

***

# Question 2

Use tensorboard visualizations of weights and learning curves (like loss and accuracy) as well to discuss about the issue raised by using zeros initializer for weights.

---

Let's see what is the impact of using normal initializer for weights.

Do not forget to send in standard deviation of gaussain distribution used for sampling weights as an argument to model constructor for all next experiments.

In [ ]:
stddev_list = [10, 1, 0.1]
for stddev in stddev_list:
    print('Normal Initializer with stddev: {}'.format(stddev))
    model = Dense(num_hidden=[200, 50, 10],
                  weight_initializer=normal_initializer,
                  bias_initializer=zero_initializer,
                  act=tf.nn.tanh,
                  logging=True,
                  stddev=stddev)

    with tf.Session() as sess:
        log_file = str(model.act.__name__) + "_" + str(model.weight_initializer.__name__) + "_" + str(stddev)
        train(x_train, y_train, x_validation, y_validation, x_test, y_test, model, sess, log_file)
    tf.reset_default_graph()

---

# Question 3

Use tensorboard visualizations of weights and learning curves (like loss and accuracy) as well to describe differences through training process caused by using different standard deviations.

---

## Different Architectures (layers and units)

Next, we will cast light on the importance of architecture of neural networks (more specifically number of layers and units)

Here we will examine two architectures. Comparing with above setting, we will increase the number of epochs to 15 and change learning rate to 0.01. We also use normal initializer for weights with `stddev=1` and `tanh` as activation function. What makes difference between two architectures is the number of layers and units.

In the first one we have 3 hidden layers with 500, 100, 50 units, respectively. This model imposes high computational cost. On the other hand, the second network has one less hidden layer but same number of units in the rest of layers. Totally it has less complexity in terms of number of parameters compared with the first network.

In [ ]:
FLAGS.num_epoch = 15
FLAGS.learning_rate = 0.01
num_hidden_list = [[500, 100, 50, 10], [100, 50, 10]]
stddev = 1
for num_hidden in num_hidden_list:
    print('Number of hidden units: ', num_hidden)
    model = Dense(num_hidden=num_hidden,
                  weight_initializer=normal_initializer,
                  bias_initializer=zero_initializer,
                  act=tf.nn.tanh,
                  logging=True,
                  stddev=stddev)
    with tf.Session() as sess:
        log_file = str(model.act.__name__) + "_" + str(model.num_hidden)
        train(x_train, y_train, x_validation, y_validation, x_test, y_test, model, sess, log_file)
    tf.reset_default_graph()

---
# Question 4
By using tensorboard visualiztions justify the different outcomes of training the above two neural network architectures.

---

## Regularization

Another important point to consider is applying regularization and being aware of its impact on training and generalization. Its most important rule is to prevent our learning process from overfitting (you may have encountered this phenomenon above!)

In first experiment L2 regularization technique is not applied. However, in rest of them, it is used for penalizing two sets of weights:

1. Weights between input and first hidden layer.
2. Weights between first and second hidden layer.

Another point to mention is that here we use `tanh` activation function for all layers.

In [ ]:
weight_decay_list = [0., 1e-4, 1e-3, 1e-2, 1e-1]
for weight_decay in weight_decay_list:
    print('Weight decay of: {}'.format(weight_decay))
    FLAGS.weight_decay = weight_decay 
    model = Dense(num_hidden=[500, 100, 50, 10],
                  weight_initializer=normal_initializer,
                  bias_initializer=zero_initializer,
                  act=tf.nn.tanh,
                  logging=True,
                  stddev=1)
    with tf.Session() as sess:
        log_file = str(model.act.__name__) + "_" + str(FLAGS.weight_decay)
        train(x_train, y_train, x_validation, y_validation, x_test, y_test, model, sess, log_file)
    tf.reset_default_graph()

---
# Question 5
By using visualiztion of learning curves (specially loss curve) try to explain the impact that L2 regularization had on training process.

---

### **Note**: 
So far, some questions were placed between cells of codes and descriptions. 

In addition to **completing the code files**, please send a **report** including your answer to these questions as well. Do not forget to put the diagrams and visualizations needed in each part.